In [1]:
# 不通过图的方式, 完成一个图基本算法的撰写
import numpy as np
from collections import deque
from typing import Optional

In [2]:
def build_adjM(vertexNum:int, edges:list[tuple[int, int]])->np.ndarray:
    vertex_id=[i for i in range(vertexNum)]
    adjM=np.zeros(shape=(vertexNum, vertexNum))

    for (u, v) in edges:
        adjM[u, v]=adjM[v, u]=1 # 无向无权图

    return adjM

In [3]:
# BFS: my version, 写得没有下面的好
# def BFS_from_certain_node(adjM, start_id, que): # 从某个结点开始 BFS
    
#     if not que:
#         que=deque()

#     vertexNum=np.shape(adjM)[0]

#     mark=np.zeros(shape=vertexNum)
#     dist=np.zeros(shape=vertexNum)
#     pred=np.full(shape=vertexNum, fill_value=-1)

#     que.append(start_id)
#     mark[start_id]=1
    
#     while que:
#         v=que.popleft()
#         mark[v]=1
        
#         # do something to v here...

#         for u in range(vertexNum):
#             if adjM[v, u]==1 and mark[u]==0:
#                 que.append(u)
#                 pred[u]=v
#                 dist[u]=dist[v]+1
#         mark[v]=2

# def BFS_Global(adjM): # 全局BFS
#     que=deque()

#     vertexNum=np.shape(adjM)[0]

#     mark=np.zeros(shape=vertexNum)
#     dist=np.zeros(shape=vertexNum)
#     pred=np.full(shape=vertexNum, fill_value=-1)

#     for v in range(vertexNum):
#         if mark[v]==0:
#             BFS_from_certain_node(adjM, start_id=v, que=[])

In [ ]:
def BFS(adjM: np.ndarray, start_id: Optional[int] = None) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    统一的BFS遍历函数,支持指定起点或全局遍历(处理非连通图)
    
    参数:
        adjM: 邻接矩阵 (nxn), 元素为0或1
        start_id: 起始节点ID(0-based),默认None表示全局遍历(处理所有连通分量)
            Optional 是 Python 标准库 typing 模块提供的「类型提示工具」.
            Optional[T] = Union[T, None]
            翻译过来就是：「参数类型可以是 T(这里是 int), 也可以是 None」.

            start_id: Optional[int] = None
                应该拆成两部分看:
                    Optional[int] 如上.
                    = None: 如果不写 start_id 参数 默认 start_id=None


            补充说明:
                Python >=3.10 中, PEP 604 引入了新语法, 支持直接用 | 表示「类型联合」, 此时可以不用 Optional:
                start_id: int | None = None  (等价于 start_id: Optional[int]=None)
            
    返回:
        mark: 标记数组 (0:未访问, 1:访问中, 2:已访问完毕)
        dist: 距离数组,起点到各节点的距离(不可达为-1)
        pred: 前驱节点数组,各节点的前驱节点ID(无前驱为-1)


    复杂度分析:
        只对一个结点v进行BFS, 复杂度 O(1+deg(v)), 其中 deg(v)是 v 的度
        全图 BFS, 复杂度 O(|V|+|E|), |V|, |E|分别表示图的顶点数,边数
    """
    vertexNum = np.shape(adjM)[0]
    if vertexNum == 0:
        return np.array([]), np.array([]), np.array([])
    
    # 初始化返回数组
    mark = np.zeros(shape=vertexNum, dtype=int)
    dist = np.full(shape=vertexNum, fill_value=-1, dtype=int)  # 不可达标记为-1更合理
    pred = np.full(shape=vertexNum, fill_value=-1, dtype=int)
    
    # 核心BFS辅助函数(内部使用)
    def _bfs_helper(start: int):

        '''
        为什么 _bfs_helper 开头有下划线?
        Python 的命名规范(PEP 8)中定义的「私有成员标记」, 
        核心目的是明确函数 / 变量的访问权限，避免外部误调用

        单下划线 _xxx:表示「内部使用的非公有成员」(约定俗成，不是强制语法限制)
        双下划线 __xxx:会触发 Python 的「名称修饰(name mangling)」, 真正限制外部访问(比如 __foo 会被改成 _类名__foo)
        '''
        que=deque()
        que.append(start)
        mark[start] = 1
        dist[start] = 0  # 起点距离为0
        
        while que:
            v = que.popleft()
            
            # TODO: 在这里添加对节点v的处理逻辑
            # 示例：print(f"Processing node {v}")
            '''
            主流代码编辑器(VS Code、PyCharm、Sublime 等)内置的「注释关键词高亮规则」
            '''
            
            # 遍历所有邻接节点
            for u in range(vertexNum):
                if adjM[v, u] == 1 and mark[u] == 0:
                    que.append(u)
                    mark[u] = 1 # 结点入队, 标记为访问中, 避免重复入队
                    pred[u] = v
                    dist[u] = dist[v] + 1
            mark[v] = 2  # 标记为已处理完毕
    
    # 执行BFS
    if start_id is not None:
        # 模式1：指定起点的BFS
        if 0 <= start_id < vertexNum:
            _bfs_helper(start_id)
        else:
            raise ValueError(f"start_id必须在[0, {vertexNum-1}]范围内")
    else:
        # 模式2：全局BFS(处理所有连通分量)
        for v in range(vertexNum):
            if mark[v] == 0:
                _bfs_helper(v)

    # 如果进行了全局BFS, 那返回的dist 应该是到各自连通分量上一点的距离
    # print(mark, dist, pred)
    return mark, dist, pred

In [5]:
'''
def DFS_from_certain_node(adjM:np.ndarray, start_id): # 从某个结点开始DFS
    vertexNum=np.shape(adjM)[0]

    mark=np.zeros(shape=vertexNum)
    pred=np.full(shape=vertexNum, fill_value=-1)
    d=np.zeros(shape=vertexNum)
    f=np.zeros(shape=vertexNum)

    time=0

    def _dfs_visit(v):
        nonlocal time

        mark[v]=1 # 标记为访问中
        time+=1
        d[v]=time

        # do something to v here...

        for u in range(vertexNum):
            if adjM[v, u]==1 and mark[u]==0:
                pred[u]=v
                _dfs_visit(u)

        mark[v]=2
        time+=1
        f[v]=time


def DFS_Global(adjM): # 全局DFS
    vertexNum=np.shape(adjM)[0]

    mark=np.zeros(shape=vertexNum)
    pred=np.full(shape=vertexNum, fill_value=-1)
    d=np.zeros(shape=vertexNum)
    f=np.zeros(shape=vertexNum)

    time=0

    def _dfs_visit(v):
        nonlocal time

        mark[v]=1
        time+=1
        d[v]=time

        # TODO: 在这里添加对节点v的处理逻辑
        # 示例：print(f"Processing node {v}")

        for u in range(vertexNum):
            if adjM[v, u]==1 and mark[u]==0:
                pred[u]=v
                _dfs_visit(u)

        mark[v]=2
        time+=1
        f[v]=time

    for v in range(vertexNum):
        if mark[v]==0:
            _dfs_visit(v)
'''


'\ndef DFS_from_certain_node(adjM:np.ndarray, start_id): # 从某个结点开始DFS\n    vertexNum=np.shape(adjM)[0]\n\n    mark=np.zeros(shape=vertexNum)\n    pred=np.full(shape=vertexNum, fill_value=-1)\n    d=np.zeros(shape=vertexNum)\n    f=np.zeros(shape=vertexNum)\n\n    time=0\n\n    def _dfs_visit(v):\n        nonlocal time\n\n        mark[v]=1 # 标记为访问中\n        time+=1\n        d[v]=time\n\n        # do something to v here...\n\n        for u in range(vertexNum):\n            if adjM[v, u]==1 and mark[u]==0:\n                pred[u]=v\n                _dfs_visit(u)\n\n        mark[v]=2\n        time+=1\n        f[v]=time\n\n\ndef DFS_Global(adjM): # 全局DFS\n    vertexNum=np.shape(adjM)[0]\n\n    mark=np.zeros(shape=vertexNum)\n    pred=np.full(shape=vertexNum, fill_value=-1)\n    d=np.zeros(shape=vertexNum)\n    f=np.zeros(shape=vertexNum)\n\n    time=0\n\n    def _dfs_visit(v):\n        nonlocal time\n\n        mark[v]=1\n        time+=1\n        d[v]=time\n\n        # TODO: 在这里添加对节点v的处理逻辑\

In [ ]:
def DFS(adjM:np.ndarray, start_id:Optional[int]=None)->tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    '''
    统一的DFS遍历函数,支持指定起点或全局遍历(处理非连通图)
    
    参数:
        adjM: 邻接矩阵 (nxn), 元素为0或1
        start_id: 起始节点ID(0-based),默认None表示全局遍历(处理所有连通分量)

    返回:
        mark: 标记数组(0:未访问, 1:访问中, 2:已访问完毕), dtype=int
        pred: 前驱节点数组,各节点的前驱节点ID(无前驱为-1), dtype=int
        d: 发现时间数组(节点首次被访问的时间戳), dtype=int; 不可达/未访问节点为0
        f: 完成时间数组(节点所有邻接节点处理完毕的时间戳), dtype=int;不可达/未访问节点为0

    复杂度分析:
        只对一个结点v进行BFS, 复杂度 O(1+deg(v)), 其中 deg(v)是 v 的度
        全图 BFS, 复杂度 O(|V|+|E|), |V|, |E|分别表示图的顶点数,边数
    '''

    vertexNum=np.shape(adjM)[0]
    if vertexNum == 0:
        return np.array([]), np.array([]), np.array([]), np.array([])

    mark=np.zeros(shape=vertexNum, dtype=int)
    pred=np.full(shape=vertexNum, fill_value=-1, dtype=int)
    d=np.zeros(shape=vertexNum, dtype=int)
    f=np.zeros(shape=vertexNum, dtype=int)

    time=0

    def _dfs_visit(v):
        nonlocal time

        mark[v]=1
        time+=1
        d[v]=time

        # TODO: 在这里添加对节点v的处理逻辑
        # 示例：print(f"Processing node {v}")

        for u in range(vertexNum):
            if adjM[v, u]==1 and mark[u]==0:
                pred[u]=v
                _dfs_visit(u) # 递归访问u

        mark[v]=2
        time+=1
        f[v]=time

    if start_id is not None:
        if 0 <= start_id <vertexNum:
            _dfs_visit(start_id)
        else:
            raise ValueError(f"start_id必须在[0, {vertexNum-1}]范围内")
    else:
        for v in range(vertexNum):
            if mark[v]==0:
                _dfs_visit(v)

    print(mark, pred, d, f)
    return mark, pred, d, f
    

In [ ]:
# test_case

VertexNum=8
Edges=[(0, 4), (0, 1), (1, 5), (2, 5), (5, 6), (2, 6), (2, 3), (3, 6), (6, 7), (3, 7)]

adjM=build_adjM(VertexNum, Edges)
BFS(adjM=adjM, start_id=1)
DFS(adjM=adjM, start_id=0)

(array([2, 2, 2, 2, 2, 2, 2, 2]),
 array([-1,  0,  5,  2,  0,  1,  3,  6]),
 array([ 1,  2,  4,  5, 14,  3,  6,  7]),
 array([16, 13, 11, 10, 15, 12,  9,  8]))